In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:')
Session = sessionmaker(bind=engine)
session = Session()


In [2]:
from datetime import datetime
from sqlalchemy import (Table, Column, Integer, Numeric, 
                        String, DateTime, ForeignKey)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

Base = declarative_base()

class Cookie(Base):
    __tablename__ = 'cookies'
    
    cookie_id = Column(Integer(), primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer())
    unit_cost = Column(Numeric(12, 2))
    
    def __repr__(self):
        return "Cookie(cookie_name='{self.cookie_name}', " \
                "cookie_recipe_url = '{self.cookie_recipe_url}', " \
                "cookie_sku='{self.cookie_sku}', " \
                "quantity={self.quantity}, " \
                "unit_cost={self.unit_cost})".format(self=self)
    
    
class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key=True)
    username = Column(String(15), nullable=False, unique=True)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, 
                       onupdate=datetime.now)
    
    def __repr__(self):
        return "User(username='{self.username}', " \
                "email_address='{self.email_address}', " \
                "phone='{self.phone}', " \
                "password='{self.password}')".format(self=self)
     
    
class Order(Base):
    __tablename__ = 'orders'
    
    order_id = Column(Integer(), primary_key=True)
    user_id = Column(Integer(), ForeignKey('users.user_id'))
    
    user = relationship('User', 
                        backref=backref('orders', order_by=order_id))
    
    def __repr__(self):
        return "Order(user_id={self.user_id}, " \
                "shipped={self.shipped})".format(self=self)
        

class LineItems(Base):
    __tablename__ = 'line_items'
    
    line_item_id = Column(Integer(), primary_key=True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    cookie_id = Column(Integer(), ForeignKey('cookies.cookie_id'))
    quantity = Column(Integer())
    extended_cost = Column(Numeric(12, 2))

    order = relationship('Order', 
                        backref=backref('line_items', 
                                        order_by=line_item_id))
    
    cookie = relationship('Cookie', uselist=False)
    
    def __repr__(self):
        return "LineItems(order_id={self.order_id}, " \
                "cookie_id={self.cookie_id}, " \
                "quantity={self.quantity}, " \
                "extended_cost={self.extended_cost})".format(self=self)

Base.metadata.create_all(engine)

In [3]:
cc_cookie = Cookie(cookie_name='chocolate chip',
                  cookie_recipe_url='http://some.aweso.me',
                  cookie_sku='CC01',
                  quantity=12,
                  unit_cost=0.50)
session.add(cc_cookie)
session.commit()

In [4]:
print(cc_cookie.cookie_id)

1


/Users/rodrigovillatoro/venv/lib/python3.5/site-packages/sqlalchemy/sql/sqltypes.py:562: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [5]:
dcc = Cookie(cookie_name='dark chocolate chip',
            cookie_recipe_url='http://dark.chocolate.chip.com',
            cookie_sku='CC02',
            quantity=1,
            unit_cost=0.75)
mol = Cookie(cookie_name='molasses',
            cookie_recipe_url='http://some.molases.com',
            cookie_sku='MOL01',
            quantity=1,
            unit_cost=0.80)

session.add(dcc)
session.add(mol)
session.flush()
print(dcc.cookie_id)
print(mol.cookie_id)

2
3


In [6]:
c1 = Cookie(cookie_name='peanut butter',
            cookie_recipe_url='http://peanut.butter.com',
            cookie_sku='PB01',
            quantity=24,
            unit_cost=0.25)
c2 = Cookie(cookie_name='oatmeal raisin',
            cookie_recipe_url='http://some.oatmeal.raisin.com',
            cookie_sku='EWW01',
            quantity=100,
            unit_cost=1.00)

session.bulk_save_objects([c1, c2])
session.commit()

# nothing will be printed because c1 and c2 objects are 
# not associated with the session. 
print(c1.cookie_id)
print(c2.cookie_id)

None
None


In [7]:
# Querying Data: get all the cookies

In [10]:
cookies = session.query(Cookie).all()
print(cookies)

[Cookie(cookie_name='chocolate chip', cookie_recipe_url = 'http://some.aweso.me', cookie_sku='CC01', quantity=12, unit_cost=0.50), Cookie(cookie_name='dark chocolate chip', cookie_recipe_url = 'http://dark.chocolate.chip.com', cookie_sku='CC02', quantity=1, unit_cost=0.75), Cookie(cookie_name='molasses', cookie_recipe_url = 'http://some.molases.com', cookie_sku='MOL01', quantity=1, unit_cost=0.80), Cookie(cookie_name='peanut butter', cookie_recipe_url = 'http://peanut.butter.com', cookie_sku='PB01', quantity=24, unit_cost=0.25), Cookie(cookie_name='oatmeal raisin', cookie_recipe_url = 'http://some.oatmeal.raisin.com', cookie_sku='EWW01', quantity=100, unit_cost=1.00)]


In [13]:
for cookie in session.query(Cookie):
    print(cookie)

Cookie(cookie_name='chocolate chip', cookie_recipe_url = 'http://some.aweso.me', cookie_sku='CC01', quantity=12, unit_cost=0.50)
Cookie(cookie_name='dark chocolate chip', cookie_recipe_url = 'http://dark.chocolate.chip.com', cookie_sku='CC02', quantity=1, unit_cost=0.75)
Cookie(cookie_name='molasses', cookie_recipe_url = 'http://some.molases.com', cookie_sku='MOL01', quantity=1, unit_cost=0.80)
Cookie(cookie_name='peanut butter', cookie_recipe_url = 'http://peanut.butter.com', cookie_sku='PB01', quantity=24, unit_cost=0.25)
Cookie(cookie_name='oatmeal raisin', cookie_recipe_url = 'http://some.oatmeal.raisin.com', cookie_sku='EWW01', quantity=100, unit_cost=1.00)


In [14]:
# Select only cookie_name and quantity
print(session.query(Cookie.cookie_name, Cookie.quantity).first())

('chocolate chip', 12)


In [16]:
# Order by quantity ascending
for cookie in session.query(Cookie).order_by(Cookie.quantity):
    print('{:3} - {}'.format(cookie.quantity, cookie.cookie_name))

  1 - dark chocolate chip
  1 - molasses
 12 - chocolate chip
 24 - peanut butter
100 - oatmeal raisin


In [17]:
# Order by quantity descending
from sqlalchemy import desc
for cookie in session.query(Cookie).order_by(desc(Cookie.quantity)):
    print('{:3} = {}'.format(cookie.quantity, cookie.cookie_name))

100 = oatmeal raisin
 24 = peanut butter
 12 = chocolate chip
  1 = dark chocolate chip
  1 = molasses
